# Proyecto (obtención dataset)

In [47]:
# --- General ---
import os
from datetime import datetime

# --- Data ---
import pandas as pd
from pandasql import sqldf
import numpy as np

# --- Conexión ---
import elasticsearch

# --- Procesamiento lenguaje: spacy ---
import spacy
# from spacy.matcher import Matcher
from spacy.matcher import PhraseMatcher

# --- Procesamiento lenguaje: gensim ---
# import gensim
# import gensim.corpora as corpora
# from gensim.utils import simple_preprocess
# from gensim.models import CoherenceModel

# --- Visualización ---
# import pyLDAvis
# import pyLDAvis.gensim_models  # don't skip this
# import matplotlib.pyplot as plt
# import geopandas as gpd
# %matplotlib inline

# ==============================================================================

# --- Configuración ---
# User password
password = os.environ.get('SOPHIA2')

# geoData
comunas = ['Ancud', 'Calbuco', 'Castro', 'Chaitén', 'Chonchi', 'Cochamó', 'Curaco de Vélez', 'Dalcahue', 'Fresia', 'Frutillar', 'Futaleufú', 'Hualaihué', 'Llanquihue', 'Los Muermos', 'Maullín', 'Osorno', 'Palena', 'Puerto Montt', 'Puerto Octay', 'Puerto Varas', 'Puqueldón', 'Purranque', 'Puyehue', 'Queilén', 'Quellón', 'Quemchi', 'Quinchao', 'Río Negro', 'San Juan de la Costa', 'San Pablo']

# --- Funciones ---
# Cargar paquete de español mediano
nlp = spacy.load("es_core_news_md")

## Inicialización peticiones Sophia2

In [48]:
# --- Nombre archivo ---
nombre_archivo = 'data_Region_X'

# --- Parameters queries ---
# País
country="chile"
# Fecha
from_="2022-01-01"
to_="2022-01-07"
# Medios de prensa
media_outlets=["elllanquihue","elaustral","laestrelladechiloe","elheraldoaustral","radiosago",
               "elrepuertero","elvacanudo","elhuemul","seminariolocal","elquellonino","elinsular",
               "radiopudeto","radioacogida","elcalbucano","segundos33",
               "prensadelestuario","fresiaahora","soychiloe"]
# Búsqueda de palabra clave
# keyword="covid"
# simple_keyword=True

# --- Data Connection ---
IP = "search.sophia2.org"
PORT = 9200
USER= "elastic"
PASS= password

# --- Connection ---
es = elasticsearch.Elasticsearch(
    IP,
    http_auth=(USER, PASS)
)

# --- Queries ---
query = { 
    "bool": { 
    "filter": [
        {"range": {
      "date": {
        "gte": from_,
        "lt": to_
      }
      }},

        { "term":  { "country": country }},
        { "terms":  { "media_outlet": media_outlets }} 
    ]
    }  
}

In [49]:
# --- Confirmación de conexión ---
res = es.search(index="news", query=query, size=10000)
print("Son %d noticias encontradas..." % res['hits']['total']['value'])

Son 248 noticias encontradas...


## Creación dataframe

In [50]:
# --- Columnas Dataframe ---
data = {'id_news':[],'country':[],'media_outlet':[],'url':[],'title':[],'text':[],'date':[]}
df = pd.DataFrame(data)  

# --- Información Dataframe---
for hit in res['hits']['hits']:

    id_news = hit['_source']['id_news']
    country = hit['_source']['country']
    media_outlet = hit['_source']['media_outlet']
    url = hit['_source']['url']
    title = hit['_source']['title']
    text = hit['_source']['text']
    date = hit['_source']['date']
    
    new_row = {'id_news':int(id_news), 'country':country, 'media_outlet':media_outlet, 'url':url, 'title':title, 'text':text, 'date':date}
    df = pd.concat([df, pd.DataFrame.from_records([new_row])])
    # df = df.append(new_row, ignore_index=True)


# Reseteamos el índice del dataframe para que sea secuencial, sino serían puros 0.
df = df.reset_index(drop=True)

# --- Elimina resiuduos ---
# Elimina duplicados
df = df.drop_duplicates(subset='url', keep='first')
# Elimina textos vacíos o con muy poco contenido
df = df[df['text'].str.len() > 50]

In [51]:
print(f"Son {len(df)} noticias encontradas que se utilizarán")
df.head()

Son 241 noticias encontradas que se utilizarán


,id_news,country,media_outlet,url,title,text,date
0,21908558.0,chile,radiosago,https://www.radiosago.cl/140-contagios-de-covi...,140 contagios de covid-19 se reportaron este j...,Así lo confirmó la autoridad sanitaria detalla...,2022-01-06
1,21908579.0,chile,radiosago,https://www.radiosago.cl/por-tormentas-electri...,Por tormentas eléctricas declaran alerta en cu...,Una Alerta Temprana Preventiva por tormentas e...,2022-01-05
2,21909623.0,chile,radiosago,https://www.radiosago.cl/gobernador-vallespin-...,Gobernador Vallespin llamó a construir entre t...,"Patricio Vallespin, gobernador regional de Los...",2022-01-01
3,21909637.0,chile,radiosago,https://www.radiosago.cl/cuatro-viviendas-fuer...,Cuatro viviendas fueron afectadas por un incen...,Un incendio se desató en la madrugada de este ...,2022-01-01
4,21909643.0,chile,radiosago,https://www.radiosago.cl/un-muerto-deja-accide...,Un muerto deja accidente la madrugada de este ...,Un hombre falleció luego que el vehículo que c...,2022-01-01


## Añadiremos las comunas mencionadas en las noticias

In [52]:
# --- Creamos Matcher utilizando nlp ( natural language processing ) ---
matcher_comunas = PhraseMatcher(nlp.vocab)
for comuna in comunas:
    matcher_comunas.add(comuna, [nlp(comuna)])

# --- Función para buscar comunas ---
for index,row in df.iterrows():
    txt = row["text"]
    
    try:
        # -- Procesamos el texto --
        doc = nlp(txt)
        matches_comunas = matcher_comunas(doc)

        # -- Creamos lista de comunas --
        for match_id, start, end in matches_comunas:
            span = doc[start:end]  # The matched span

    except:
        pass

In [53]:
# --- Reemplazamos los espacios por guiones bajos, para cada Comuna ---
# ejemplo: "San Juan de la Costa" -> "San_Juan_de_la_Costa"
df_comunas = df
for comuna in comunas:
    df_comunas.insert(7,comuna.replace(" ","_"),0)

# --- Insertamos las comunas al dataframe (df) ---
for index,row in df_comunas.iterrows():
    txt = row["text"]
    
    try:
    
        doc = nlp(txt)
        matches_comunas = matcher_comunas(doc)

        for match_id, start, end in matches_comunas:
            span = doc[start:end]  # The matched span
            df_comunas.at[index,span.text.replace(" ","_")]=1
    except Exception as e:
        print(e)
        pass

## Guardamos la información CSV

In [54]:
# --- Info Archivo ---
archivo= nombre_archivo+"_"+from_+"_"+to_+".csv"
print("El archivo se llama:")
print(archivo)

# --- Guarda archivo ---
df_comunas.to_csv("./data/"+archivo)

El archivo se llama:
data_Region_X.csv


###  Con esto tenemos toda la información necesaria para cargar los datos y utilizarlos.